In [13]:
from pymongo import MongoClient
import json
from dotenv import load_dotenv
import os

load_dotenv()
uri = os.getenv("MONGODB_URI")

client = MongoClient(uri)
db = client["bookstore"]
collection = db["customers"]

with open("collection.json", "r") as f:
    data = json.load(f)
collection.insert_many(data)


InsertManyResult([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], acknowledged=True)

In [16]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from pymongo import MongoClient
import os
import json

load_dotenv()

# 🔑 Load keys
MONGO_URI = os.getenv("MONGODB_URI")
OPENROUTER_KEY = os.getenv("API_KEY")  

# ✅ Connect to MongoDB
client = MongoClient(MONGO_URI)
collection = client["bookstore"]["customers"]

# ✅ Set up LLM
llm = ChatOpenAI(
    model="mistralai/mixtral-8x7b-instruct",
    temperature=0,
    openai_api_key=OPENROUTER_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

# ✅ Build prompt to get aggregation pipeline
def ask_llm_for_pipeline(user_question):
    prompt = f"""
You are a MongoDB expert. The 'customers' collection has this schema:
Each customer document may contain an "Orders" array.
Each "Order" contains "OrderDetails", which is an array of books with fields:
- Title
- Quantity
- Price
- Subtotal

Given this structure, generate a MongoDB aggregation pipeline to answer:
"{user_question}"

Return only a valid JSON array. Do not include any explanation or comments.
"""
    response = llm.predict(prompt)
    return response.strip()
# ✅ Execute the pipeline
def run_pipeline(pipeline_json_str):
    try:
        pipeline = json.loads(pipeline_json_str)
        result = list(collection.aggregate(pipeline))
        return result
    except Exception as e:
        return f"❌ Error: {str(e)}"




In [19]:
# 🔎 Sample test
user_question = "What are the top 5 books by total revenue?"
pipeline_str = ask_llm_for_pipeline(user_question)
print("Generated Pipeline:\n", pipeline_str)

results = run_pipeline(pipeline_str)
print("\nResults:\n", results)

Generated Pipeline:
 [
  {
    "$unwind": "$Orders"
  },
  {
    "$unwind": "$Orders.OrderDetails"
  },
  {
    "$group": {
      "_id": "$Orders.OrderDetails.Title",
      "totalRevenue": {
        "$sum": "$Orders.OrderDetails.Subtotal"
      }
    }
  },
  {
    "$sort": {
      "totalRevenue": -1
    }
  },
  {
    "$limit": 5
  },
  {
    "$project": {
      "_id": 0,
      "Title": "$_id",
      "TotalRevenue": "$totalRevenue"
    }
  }
]

Results:
 [{'Title': 'Subject development.', 'TotalRevenue': 1138.88}, {'Title': 'Film possible enough anyone.', 'TotalRevenue': 968.99}, {'Title': 'Impact second.', 'TotalRevenue': 854.28}, {'Title': 'Last music.', 'TotalRevenue': 793.9200000000001}, {'Title': 'Research edge.', 'TotalRevenue': 769.04}]
